In [56]:
import pandas as pd

Loading train and test data

In [57]:
fire=pd.read_csv("/Users/da-m1-09/Downloads/Train (4).csv")

In [58]:
test=pd.read_csv("/Users/da-m1-09/Downloads/Test (1).csv")

In [59]:

from sklearn.preprocessing import StandardScaler

In [60]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
from xgboost import XGBRFRegressor

In [61]:
from sklearn.model_selection import RandomizedSearchCV


In [62]:
fire

,ID,lat,lon,burn_area,climate_aet,climate_def,climate_pdsi,climate_pet,climate_pr,climate_ro,...,landcover_0,landcover_1,landcover_2,landcover_3,landcover_4,landcover_5,landcover_6,landcover_7,landcover_8,precipitation
0,0_2001-01-01,-15.858835,29.237029,0.0,1195,0,263,1195,206,10,...,0.0,0.0,0.018654,0.0,0.714446,0.012174,0.244890,0.009836,0.0,0.256932
1,1_2001-01-01,-15.858835,29.487029,0.0,1196,0,232,1196,201,10,...,0.0,0.0,0.000000,0.0,0.654783,0.000095,0.345121,0.000000,0.0,0.273093
2,2_2001-01-01,-15.858835,29.737029,0.0,1190,0,314,1190,192,10,...,0.0,0.0,0.000000,0.0,0.516421,0.000000,0.483579,0.000000,0.0,0.285109
3,3_2001-01-01,-15.858835,29.987029,0.0,1144,0,321,1144,186,66,...,0.0,0.0,0.000000,0.0,0.299000,0.163902,0.537098,0.000000,0.0,0.298418
4,4_2001-01-01,-15.858835,30.237029,0.0,1187,0,413,1187,186,9,...,0.0,0.0,0.000000,0.0,0.277392,0.067742,0.654866,0.000000,0.0,0.315621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83143,528_2013-12-01,-22.108835,30.737029,0.0,1210,324,142,1535,127,6,...,0.0,0.0,0.000000,0.0,0.008768,0.000000,0.991232,0.000000,0.0,0.132015
83144,529_2013-12-01,-22.108835,30.987029,0.0,1295,195,126,1490,136,7,...,0.0,0.0,0.000000,0.0,0.216560,0.000000,0.783440,0.000000,0.0,0.124280
83145,530_2013-12-01,-22.108835,31.237029,0.0,1254,272,111,1527,132,7,...,0.0,0.0,0.000000,0.0,0.392621,0.000000,0.607379,0.000000,0.0,0.115464
83146,531_2013-12-01,-22.108835,31.487029,0.0,1221,289,68,1510,128,6,...,0.0,0.0,0.000000,0.0,0.637696,0.000000,0.362304,0.000000,0.0,0.119895


Feature Engineering-Creating a 'Year' col

In [63]:
fire['year'] = fire['ID'].str.split('_', expand=True)[1].str[:4]

In [64]:
fire

,ID,lat,lon,burn_area,climate_aet,climate_def,climate_pdsi,climate_pet,climate_pr,climate_ro,...,landcover_1,landcover_2,landcover_3,landcover_4,landcover_5,landcover_6,landcover_7,landcover_8,precipitation,year
0,0_2001-01-01,-15.858835,29.237029,0.0,1195,0,263,1195,206,10,...,0.0,0.018654,0.0,0.714446,0.012174,0.244890,0.009836,0.0,0.256932,2001
1,1_2001-01-01,-15.858835,29.487029,0.0,1196,0,232,1196,201,10,...,0.0,0.000000,0.0,0.654783,0.000095,0.345121,0.000000,0.0,0.273093,2001
2,2_2001-01-01,-15.858835,29.737029,0.0,1190,0,314,1190,192,10,...,0.0,0.000000,0.0,0.516421,0.000000,0.483579,0.000000,0.0,0.285109,2001
3,3_2001-01-01,-15.858835,29.987029,0.0,1144,0,321,1144,186,66,...,0.0,0.000000,0.0,0.299000,0.163902,0.537098,0.000000,0.0,0.298418,2001
4,4_2001-01-01,-15.858835,30.237029,0.0,1187,0,413,1187,186,9,...,0.0,0.000000,0.0,0.277392,0.067742,0.654866,0.000000,0.0,0.315621,2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83143,528_2013-12-01,-22.108835,30.737029,0.0,1210,324,142,1535,127,6,...,0.0,0.000000,0.0,0.008768,0.000000,0.991232,0.000000,0.0,0.132015,2013
83144,529_2013-12-01,-22.108835,30.987029,0.0,1295,195,126,1490,136,7,...,0.0,0.000000,0.0,0.216560,0.000000,0.783440,0.000000,0.0,0.124280,2013
83145,530_2013-12-01,-22.108835,31.237029,0.0,1254,272,111,1527,132,7,...,0.0,0.000000,0.0,0.392621,0.000000,0.607379,0.000000,0.0,0.115464,2013
83146,531_2013-12-01,-22.108835,31.487029,0.0,1221,289,68,1510,128,6,...,0.0,0.000000,0.0,0.637696,0.000000,0.362304,0.000000,0.0,0.119895,2013


In [65]:
fire.columns

Index(['ID', 'lat', 'lon', 'burn_area', 'climate_aet', 'climate_def',
       'climate_pdsi', 'climate_pet', 'climate_pr', 'climate_ro',
       'climate_soil', 'climate_srad', 'climate_swe', 'climate_tmmn',
       'climate_tmmx', 'climate_vap', 'climate_vpd', 'climate_vs', 'elevation',
       'landcover_0', 'landcover_1', 'landcover_2', 'landcover_3',
       'landcover_4', 'landcover_5', 'landcover_6', 'landcover_7',
       'landcover_8', 'precipitation', 'year'],
      dtype='object')

In [66]:
fire['elevation'].value_counts()

elevation
413.474762     156
1045.114480    156
1160.326085    156
1061.769676    156
1083.587046    156
              ... 
1197.977194    156
1137.913013    156
1067.824644    156
968.529723     156
265.379418     156
Name: count, Length: 533, dtype: int64

In [67]:
test['year'] = test['ID'].str.split('_', expand=True)[1].str[:4]

In [68]:
test

,ID,lat,lon,climate_aet,climate_def,climate_pdsi,climate_pet,climate_pr,climate_ro,climate_soil,...,landcover_1,landcover_2,landcover_3,landcover_4,landcover_5,landcover_6,landcover_7,landcover_8,precipitation,year
0,0_2014-01-01,-15.858835,29.237029,1146,74,-451,1220,120,6,212,...,0.000615,0.015418,0.0,0.416704,0.023724,0.530016,0.013524,0.0,0.404243,2014
1,1_2014-01-01,-15.858835,29.487029,1118,98,-405,1216,117,6,330,...,0.000000,0.000000,0.0,0.406436,0.000446,0.593118,0.000000,0.0,0.421489,2014
2,2_2014-01-01,-15.858835,29.737029,1067,137,-468,1204,111,6,194,...,0.000000,0.000000,0.0,0.254791,0.000000,0.745209,0.000000,0.0,0.419122,2014
3,3_2014-01-01,-15.858835,29.987029,969,187,-533,1156,101,5,78,...,0.000000,0.000000,0.0,0.219340,0.182153,0.598507,0.000000,0.0,0.404596,2014
4,4_2014-01-01,-15.858835,30.237029,927,270,-505,1197,96,5,139,...,0.000000,0.000000,0.0,0.220239,0.097748,0.682013,0.000000,0.0,0.371100,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25579,528_2017-12-01,-22.108835,30.737029,802,1030,168,1832,84,4,81,...,0.000000,0.000000,0.0,0.029888,0.000000,0.970112,0.000000,0.0,0.049016,2017
25580,529_2017-12-01,-22.108835,30.987029,819,971,169,1790,85,4,81,...,0.000000,0.000000,0.0,0.396455,0.000000,0.603545,0.000000,0.0,0.041220,2017
25581,530_2017-12-01,-22.108835,31.237029,756,1076,156,1833,79,4,88,...,0.000000,0.000000,0.0,0.455142,0.000000,0.544858,0.000000,0.0,0.031323,2017
25582,531_2017-12-01,-22.108835,31.487029,749,1071,178,1820,78,4,102,...,0.000000,0.000000,0.0,0.680874,0.000000,0.319126,0.000000,0.0,0.032025,2017


In [69]:
test['landcover_8'].value_counts()

landcover_8
0.000000    22417
0.000615       96
0.005532       71
0.001291       48
0.137395       48
            ...  
0.023669        1
0.001344        1
0.004610        1
0.301823        1
0.056678        1
Name: count, Length: 81, dtype: int64

In [70]:

test = test.drop(['climate_swe','ID'],axis=1)

In [71]:
X=fire.drop(['burn_area','climate_swe','ID'],axis=1)

In [72]:
y=fire[['burn_area']]

In [73]:
scaler=StandardScaler()

In [74]:
x_train=scaler.fit_transform(X)


In [75]:
x_test=scaler.transform(test)

In [76]:
modelxgb = xgb.XGBRegressor(random_state=101)

#Defining the parameters
param_distributions = {
    'learning_rate': [0.001, 0.005, 0.01, 0.02],
    'n_estimators': [300, 500, 700,1000],
    'max_depth': [3, 6, 9],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'gamma': [0, 0.1, 0.5],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [1, 1.5, 2.0]
}

In [77]:
xgb_regressor = xgb.XGBRegressor(objective='reg:squarederror', n_jobs=-1)

random_search = RandomizedSearchCV(
    estimator=xgb_regressor,
    param_distributions=param_distributions,
    n_iter=50,
    scoring='neg_mean_squared_error',
    cv=5,
    n_jobs=-1,
    random_state=42
)


# Scale features


In [78]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)

# model fitting


In [79]:

random_search.fit(X_train_scaled,y)

RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                          random_state=None, ...),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.7, 0.8, 0.9],
                                        'gamma': [0, 0.1, 0.5],
                                        'learning_rate': [0.001, 0.005, 0.01,
                                                          0.02],
                                        'max_depth': [3, 6, 9],
                                        'min_child_weight': [1, 3, 5],
                                        'n_estimators': [300, 500, 700, 1000],
                                        'reg_alpha': [0, 0.1, 0.5],
                                        'reg_lambda': [1, 1.5, 2.0],
                                        'subsample': [0.7, 0.8, 0.9]},
                   random_state=42, scoring='neg_mean_squared_error')

# Print the best parameters and the corresponding RMSE

In [91]:

print("Best parameters:", random_search.best_params_)
best_model = random_search.best_estimator_


Best parameters: {'subsample': 0.9, 'reg_lambda': 1, 'reg_alpha': 0.5, 'n_estimators': 500, 'min_child_weight': 3, 'max_depth': 9, 'learning_rate': 0.01, 'gamma': 0, 'colsample_bytree': 0.7}


#getting the predictions

In [92]:

best_predictions = best_model.predict(X_test_scaled)
best_predictions


array([0.00104788, 0.00018664, 0.00122566, ..., 0.00270281, 0.00394933,
       0.0061019 ], dtype=float32)

Preparing the Submission File

In [93]:
sub=pd.DataFrame(best_predictions)

In [94]:
sub

,0
0,0.001048
1,0.000187
2,0.001226
3,0.001361
4,0.001452
...,...
25579,0.001462
25580,0.000527
25581,0.002703
25582,0.003949


In [95]:
sub.isnull().sum()

0    0
dtype: int64

Reading the test data

In [96]:
data=pd.read_csv("/Users/da-m1-09/Downloads/Test (1).csv")

In [97]:
data['Burn Area']=sub

In [98]:
df=data[['ID','Burn Area']]

In [99]:
df.to_csv('submission_test.csv', index=False)

In [89]:
df

,ID,Burn Area
0,0_2014-01-01,0.001048
1,1_2014-01-01,0.000187
2,2_2014-01-01,0.001226
3,3_2014-01-01,0.001361
4,4_2014-01-01,0.001452
...,...,...
25579,528_2017-12-01,0.001462
25580,529_2017-12-01,0.000527
25581,530_2017-12-01,0.002703
25582,531_2017-12-01,0.003949


In [90]:
import pickle

# Save the trained model to a file
with open('xgb_model.pkl', 'wb') as file:
    pickle.dump(best_model, file)
